In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import geopandas as gpd 
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sys
sys.path.append("../") 

from utils.paths import make_dir_line

modality = 'u'
project = 'Ciencia de los datos'
data = make_dir_line(modality, project)

raw = data('raw')
processed = data('processed')

In [ ]:
ter = pd.read_parquet(processed / 'ter.parquet.gzip')
ter.rename(columns={'Territory':'DEN_RIP'}, inplace=True)
# ter['DEN_RIP'] = ter['DEN_RIP'].str.title()
ter.head()

In [ ]:
chunk = pd.read_parquet(processed / 'chunk.parquet.gzip')
chunk.head()

In [ ]:
geo = gpd.read_file(raw / "Limiti01012023_g/RipGeo01012023_g/RipGeo01012023_g_WGS84.shp")
geo.geometry = geo.geometry.to_crs(epsg = 4326)
geo.head()

In [ ]:
geo.head(10)

In [ ]:
lista_verificacion = geo['DEN_RIP'].unique()
print(lista_verificacion)

In [ ]:
df_incluidos = ter[ter['DEN_RIP'].isin(lista_verificacion)]
df_incluidos.head()

In [ ]:
df_results = pd.merge(chunk, df_incluidos, on=['ITTER107'])
print(df_results.shape)

In [ ]:
df_results.head()

In [ ]:
df_results = df_results.loc[:,['ITTER107','DEN_RIP','SEXISTAT1','Value']]
df_results = df_results.groupby(['ITTER107', 'DEN_RIP', 'SEXISTAT1'], as_index=False).sum()
df_results.head()

In [ ]:
print(df_results.shape, geo.shape)
geo_data = pd.merge(df_results, geo, on=['DEN_RIP'])
print(geo_data.shape)

In [ ]:
geo_data = gpd.GeoDataFrame(geo_data, geometry = geo_data.geometry)
geo_data.geometry = geo_data.geometry.to_crs(epsg = 4326)
print(type(geo_data))
geo_data.head()

In [ ]:
geo_data.to_file(processed / 'geo_data.geojson', driver='GeoJSON')  

In [ ]:
print('Vane y Oscar')